In [207]:
#coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import random 
from sklearn import datasets 

In [208]:
# 数据预处理
def getTrainSet():
    
    iris = datasets.load_iris() 
    data = iris.data[:, :4]         #取特征空间中的4个维度(纯数据)
    
    return data

# 欧式距离计算方法
def get_distance(value1, value2):
    sum = 0
    for x,y in zip(value1, value2):
        sum += (x-y) ** 2
    distance = sum ** 0.5
    
    return distance

# 不断迭代减少Omiga集合中元素
def find_cluster(new_L, L_detail):
    index = random.randint(0,len(new_L)-1)           #获取随机种子index
    sub = L_detail[index]
    ret = list(set(new_L).intersection(set(L_detail[index])))    # 选出在L中又在Ci中的核心对象
    for i in ret:
        del L_detail[new_L.index(i)]
        new_L.remove(i)

    return sub, new_L

# 测试

In [209]:
def test(e, minpts):  
#1. 数据预处理
    data = getTrainSet()
    
#2. 获得核心对象集合
    # 得出每个点到任一点的欧式距离集合
    distance_list = []
    for d in data:
        d_list = []
        for each in data: 
            distance = get_distance(each,d)
            d_list.append(distance)
        distance_list.append(d_list)
    #print(distance_list)                        # distance_list = [[0,0.1,...], [0.3,0,...],....,[..,0.9,0]]

# 3. 获得初始核心对象总表L、核心对象聚类簇L_detail
    L = []                     #核心对象总表 L
    L_detail = []              #核心对象详细聚类簇总表 L_detail

    for item in distance_list:
        tmp = [item.index(a) for a in item if a < e]          #1-距离e以内
        tmp = list(set(tmp))

        if len(tmp) >= minpts:                                #2-个数mintps以上    
            for t in tmp:
                if t not in L:
                    L.append(t)
                    L_detail.append(tmp)          #核心对象的聚类簇添加进去   
        
# 4. 获得聚类结果
    new_L = L.copy()
    unvisited = []              # 未访问下标 unvisited
    for i in range(150):
        unvisited.append(i)
    cluster_list = []
    while new_L != []:
        cluster, new_L = find_cluster(new_L, L_detail)
        cover = list(set(unvisited).intersection(set(cluster)))
        for c in cover:
            unvisited.remove(c)    
        cluster_list.append(cover)

    print(cluster_list)
    
    return unvisited,cluster_list

# 性能测试 & 寻找最优参数

In [205]:
# 遍历：求最佳参数(e,minpts)

judge_dict = {}
for x in range(3,10):
    for y in range(3,10):
        unvisited,cluster_list = test(0.1 * x , y)
        count = 0
        for cc in cluster_list:
            count += len(cc)
        judge = len(unvisited)/len(data)*100 + len(cluster_list)
        judge_dict[str(x*0.1)+'-'+str(y)] = judge
        
print(min(judge_dict, key=judge_dict.get))


0.9-4


In [210]:
#最终测试

unvisited,cluster_list = test(0.9,4)

count = 0
for cc in cluster_list:
    count += len(cc)

print('Noice Rate:',len(unvisited)/len(data)*100,'%')
print('Cluster No:',len(cluster_list))

[[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 46, 47, 48, 49], [65, 74, 75, 76, 77, 110, 50, 51, 52, 54, 86, 56, 58], [133, 134, 138, 146, 147, 149, 55, 63, 68, 70, 72, 73, 78, 83, 87, 91, 97, 101, 103, 111, 113, 116, 119, 123, 126, 127], [130, 131, 102, 135, 105, 107, 117, 118, 122, 125], [53, 59, 60, 61, 62, 64, 66, 67, 69, 71, 79, 80, 81, 88, 89, 90, 92, 94, 95, 96, 99, 106], [8, 41, 13, 45, 22, 24, 28], [128, 100, 132, 104, 136, 137, 140, 143, 112, 144, 115, 148, 120, 124], [121, 114, 84, 85], [82], [145, 139, 141], [57, 98, 93], [129, 108, 109], [15]]
Noice Rate: 0.6666666666666667 %
Cluster No: 13
